# Model_X (version 3)

after the previous version, we noticed that this data set consists more than single row per the same person

# 1. Setup and Data Loading

In [2]:
# --- 1. Install Necessary Libraries ---
# (Run these lines once if you don't have them in your new environment)
# !pip install xgboost catboost shap

# --- 2. Import All Libraries ---
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap

# Preprocessing tools
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Model building and evaluation
# --- We now import GroupShuffleSplit for the patient-aware split ---
from sklearn.model_selection import train_test_split, GroupShuffleSplit
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier, Pool

# --- 3. Load Data ---
# IMPORTANT: Update this path to where your CSV file is located
DATA_FILE_PATH = './Dataset/Dementia Prediction Dataset.csv' 
TARGET_COLUMN = 'DEMENTED'
GROUP_COLUMN = 'NACCID'  # This is the Patient ID

try:
    # Use low_memory=False to avoid DtypeWarning during dtype inference.
    data = pd.read_csv(DATA_FILE_PATH, low_memory=False)
    print(f"Successfully loaded data. Shape: {data.shape}")
    print(f"Target column '{TARGET_COLUMN}' found.")
    print(f"Group column '{GROUP_COLUMN}' found.")
except (FileNotFoundError, KeyError):
    print(f"Error: Could not load data or find all necessary columns.")
    print("Please check DATA_FILE_PATH, TARGET_COLUMN, and GROUP_COLUMN variables.")
    # Create a small dummy dataset for demonstration
    data = pd.DataFrame({
        'NACCID': ['A', 'A', 'B', 'C', 'C', 'C', 'D', 'E', 'E', 'F'],
        'NACCAGE': [65, 66, 70, 80, 81, 82, 75, 999, 68, 69], 
        'SEX': [1, 1, 2, 1, 1, 1, 2, 1, 2, 1], 
        'EDUC': [12, 12, 16, 8, 8, 8, 20, 99, 14, 16],
        'MARISTAT': [1, 1, 2, 1, 1, 5, 5, 9, 2, 1], 
        'RACE': [1, 1, 1, 2, 2, 2, 5, 99, 1, 1], 
        'INRELTO': [1, 1, 2, 9, 3, 3, 3, 1, 1, 2],
        'CVHATT': [0, 0, 1, 2, 2, 2, 0, 9, 0, 1], 
        'CBSTROKE': [0, 0, 0, 1, 1, 1, 0, 9, 0, 0], 
        'DIABETES': [0, 1, 2, 1, 1, 1, 0, 9, 1, 0],
        'HYPERTEN': [1, 1, 1, 0, 0, 0, 0, 9, 1, 1], 
        'HYPERCHO': [1, 1, 0, 2, 2, 9, 0, 0, 1, 0], 
        'TBI': [0, 0, 0, 0, 0, 0, 0, 9, 1, 0], 
        'DEP2YRS': [0, 0, 1, 0, 0, 0, 0, 9, 1, 0], 
        'NACCBMI': [25.1, 25.5, 28.9, 32.0, 32.1, 32.2, 22.4, 888.8, 26.0, 27.0],
        'DEMENTED': [0, 0, 1, 1, 1, 1, 0, 1, 0, 1]
    })
    print(f"Loaded dummy data for demonstration. Shape: {data.shape}")

Successfully loaded data. Shape: (195196, 1024)
Target column 'DEMENTED' found.
Group column 'NACCID' found.


# 2. Preprocessing and Feature Engineering

In [3]:
# --- 1. Define Feature Sets ---
# Based on hackathon rules: A1, A2, A5, B1 are allowed.

# Numerical features that will be imputed (median) and scaled
numeric_features = [
    'NACCAGE',  # Subject's age
    'EDUC',     # Subject's years of education
    'NACCBMI'   # Subject's Body Mass Index
]

# Categorical features that will be imputed (most frequent) and one-hot encoded
categorical_features = [
    'SEX',      # Subject's sex
    'MARISTAT', # Marital status
    'RACE',     # Subject's race
    'INRELTO'   # Co-participant's relationship to subject
]

# Self-reported health history (Form A5)
# We will binarize these and engineer a new feature
health_history_features = [
    'CVHATT',   # Heart attack [cite: 32]
    'CBSTROKE', # Stroke [cite: 32]
    'DIABETES', # Diabetes [cite: 34]
    'HYPERTEN', # Hypertension [cite: 34]
    'HYPERCHO', # Hypercholesterolemia [cite: 34]
    'TBI',      # Traumatic Brain Injury [cite: 34]
    'DEP2YRS'   # Depression in last 2 years [cite: 36]
]

# --- 2. Clean Missing/Unknown Values ---
# We replace all non-standard "missing" or "unknown" codes with np.nan
# This is critical for scikit-learn's imputers to work.
missing_values_map = {
    'NACCAGE': [999],           # [cite: 420]
    'EDUC': [99],               # [cite: 367]
    'NACCBMI': [888.8, 888],    # [cite: 1118]
    'MARISTAT': [9],            # [cite: 377]
    'RACE': [99],               # [cite: 295]
    'INRELTO': [9],             # [cite: 588]
    # For Form A5, 9 means "Unknown"
    'CVHATT': [9], 'CBSTROKE': [9], 'DIABETES': [9], 'HYPERTEN': [9], # [cite: 1094, 1098, 1105, 1106]
    'HYPERCHO': [9], 'TBI': [9], 'DEP2YRS': [9] # [cite: 1107, 1102, 1115]
}

for col, missing_vals in missing_values_map.items():
    if col in data.columns:
        data[col] = data[col].replace(missing_vals, np.nan)

print("Replaced special 'missing' codes with NaN.")

# --- 3. Feature Engineering ---
# 3a. Binarize Health History
# We map 0=Absent to 0, and 1=Recent/Active or 2=Remote/Inactive to 1 (Present).
for col in health_history_features:
    if col in data.columns:
        data[col] = data[col].map({0: 0, 1: 1, 2: 1})

# 3b. Create 'ComorbidityCount'
# This counts how many conditions are present.
# We fill NaNs with 0 (assuming 'unknown' means 'absent') before summing.
data['ComorbidityCount'] = data[health_history_features].fillna(0).sum(axis=1)

# Add our new engineered feature to the numeric list
numeric_features.append('ComorbidityCount')
print("Engineered 'ComorbidityCount' feature.")

# --- 4. Separate Features (X) and Target (y) ---
if TARGET_COLUMN not in data.columns:
    print(f"FATAL ERROR: Target column '{TARGET_COLUMN}' not found!")
else:
    # X contains the raw features to be processed
    X = data[numeric_features + categorical_features]
    # y contains the final target
    y = data[TARGET_COLUMN]
    # groups contains the Patient IDs for the split
    groups = data[GROUP_COLUMN]

    # --- 5. Define Preprocessing Pipelines ---
    numeric_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ])

    categorical_pipeline = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
    ])

    # --- 6. Create the ColumnTransformer ---
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_pipeline, numeric_features),
            ('cat', categorical_pipeline, categorical_features)
        ],
        remainder='drop' # Drop any columns we didn't explicitly select
    )

    # --- 7. Apply Preprocessing ---
    # Fit and transform the entire dataset to prepare it
    X_processed = preprocessor.fit_transform(X)

    # Get the new feature names (e.g., after one-hot encoding)
    feature_names = preprocessor.get_feature_names_out()
    
    # Convert the processed data back to a DataFrame (useful for SHAP later)
    X_processed_df = pd.DataFrame(X_processed, columns=feature_names)

    print("\n--- Preprocessing Complete ---")
    print(f"Shape of processed features (X): {X_processed.shape}")
    print(f"Shape of target (y): {y.shape}")
    print(f"Shape of groups (groups): {groups.shape}")
    print("\nProcessed features (first 5 rows):")
    print(X_processed_df.head())

Replaced special 'missing' codes with NaN.
Engineered 'ComorbidityCount' feature.

--- Preprocessing Complete ---
Shape of processed features (X): (195196, 26)
Shape of target (y): (195196,)
Shape of groups (groups): (195196,)

Processed features (first 5 rows):
   num__NACCAGE  num__EDUC  num__NACCBMI  num__ComorbidityCount  cat__SEX_1.0  \
0     -0.424061   0.136566      0.831051               1.549393           1.0   
1     -0.327002   0.136566      0.683279              -0.858180           1.0   
2     -0.812297   0.136566      0.074805              -0.858180           1.0   
3     -1.103474   0.136566      0.222577              -0.858180           0.0   
4      0.255351  -1.107783     -0.333743              -0.858180           1.0   

   cat__SEX_2.0  cat__MARISTAT_1.0  cat__MARISTAT_2.0  cat__MARISTAT_3.0  \
0           0.0                1.0                0.0                0.0   
1           0.0                1.0                0.0                0.0   
2           0.0       

# 3. Patient-Aware data Splitting

We are now using GroupShuffleSplit to ensure that all records (visits) for any given patient (NACCID) are kept together in either the training set or the testing set, but never split between both.

This prevents data leakage

In [4]:
# --- 1. Set up the GroupShuffleSplit ---
# This splitter ensures that all rows for a single 'group' (NACCID)
# stay in the same set (either train or test).

print("--- Preparing Patient-Aware (Grouped) Split ---")

# We create one split (n_splits=1) with 20% of the *groups* (patients) 
# held out for the test set.
gss = GroupShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

# gss.split() returns the *indices* of the rows for train/test
# We need to get the first (and only) split from the generator
train_idx, test_idx = next(gss.split(X_processed, y, groups=groups))

# --- 2. Create the Training and Testing Sets ---
# We use the indices to select the correct rows from our processed data

X_train = X_processed[train_idx]
y_train = y.iloc[train_idx]  # Use .iloc for pandas Series

X_test = X_processed[test_idx]
y_test = y.iloc[test_idx]    # Use .iloc for pandas Series

# --- 3. Report on the Split ---
print("--- Data Splitting Complete ---")
print(f"Total rows:      {len(X_processed)}")
print(f"Training rows:   {len(X_train)}")
print(f"Testing rows:    {len(X_test)}")
print("\nChecking for patient overlap (data leakage)...")

train_patients = set(groups.iloc[train_idx])
test_patients = set(groups.iloc[test_idx])
overlap = train_patients.intersection(test_patients)

if len(overlap) == 0:
    print("SUCCESS: No patients are shared between train and test sets.")
    print(f"Total patients:    {len(train_patients) + len(test_patients)}")
    print(f"Training patients: {len(train_patients)}")
    print(f"Testing patients:  {len(test_patients)}")
else:
    print(f"ERROR: {len(overlap)} patients are in BOTH train and test sets.")

--- Preparing Patient-Aware (Grouped) Split ---
--- Data Splitting Complete ---
Total rows:      195196
Training rows:   155758
Testing rows:    39438

Checking for patient overlap (data leakage)...
SUCCESS: No patients are shared between train and test sets.
Total patients:    52537
Training patients: 42029
Testing patients:  10508


# 4. Model Development & Comparison

re-run the all four models we have used in previous versions (in 1 & 2 notebooks)

In [ ]:
# --- 1. Train Baseline Model: Logistic Regression ---
print("--- Training Logistic Regression (Patient-Aware Split) ---")
lr_model = LogisticRegression(random_state=42, max_iter=1000)
lr_model.fit(X_train, y_train)
lr_preds = lr_model.predict(X_test)
lr_accuracy = accuracy_score(y_test, lr_preds)

print(f"Logistic Regression Accuracy: {lr_accuracy:.4f}")
print(classification_report(y_test, lr_preds))
print("-" * 40)

# --- 2. Train Random Forest Classifier ---
print("--- Training Random Forest (Patient-Aware Split) ---")
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_preds)

print(f"Random Forest Accuracy: {rf_accuracy:.4f}")
print(classification_report(y_test, rf_preds))
print("-" * 40)

# --- 3. Train XGBoost Classifier ---
print("--- Training XGBoost Classifier (Patient-Aware Split) ---")
xgb_model = XGBClassifier(
    random_state=42, 
    eval_metric='logloss'
)
xgb_model.fit(X_train, y_train)
xgb_preds = xgb_model.predict(X_test)
xgb_accuracy = accuracy_score(y_test, xgb_preds)

print(f"XGBoost Accuracy: {xgb_accuracy:.4f}")
print(classification_report(y_test, xgb_preds))
print("-" * 40)

# --- 4. Train CatBoost Classifier ---
print("--- Training CatBoost Classifier (Patient-Aware Split) ---")
cat_model = CatBoostClassifier(
    random_seed=42,
    verbose=0  # Suppress training output
)
cat_model.fit(X_train, y_train)
cat_preds = cat_model.predict(X_test)
cat_accuracy = accuracy_score(y_test, cat_preds)

print(f"CatBoost Accuracy: {cat_accuracy:.4f}")
print(classification_report(y_test, cat_preds))
print("-" * 40)

# --- 5. Final Model Comparison (Patient-Aware) ---
print("\n--- 🏆 Model Performance Summary (Patient-Aware) ---")
print(f"Logistic Regression: {lr_accuracy:.4f}")
print(f"Random Forest:       {rf_accuracy:.4f}")
print(f"XGBoost:             {xgb_accuracy:.4f}")
print(f"CatBoost:            {cat_accuracy:.4f}")

--- Training Logistic Regression (Patient-Aware Split) ---
Logistic Regression Accuracy: 0.7287
              precision    recall  f1-score   support

           0       0.75      0.94      0.83     28023
           1       0.59      0.21      0.31     11415

    accuracy                           0.73     39438
   macro avg       0.67      0.58      0.57     39438
weighted avg       0.70      0.73      0.68     39438

----------------------------------------
--- Training Random Forest (Patient-Aware Split) ---
Random Forest Accuracy: 0.6932
              precision    recall  f1-score   support

           0       0.76      0.82      0.79     28023
           1       0.46      0.38      0.42     11415

    accuracy                           0.69     39438
   macro avg       0.61      0.60      0.60     39438
weighted avg       0.68      0.69      0.68     39438

----------------------------------------
--- Training XGBoost Classifier (Patient-Aware Split) ---


/opt/anaconda3/envs/metal/lib/python3.13/site-packages/xgboost/training.py:183: UserWarning: [18:45:01] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


XGBoost Accuracy: 0.7369
              precision    recall  f1-score   support

           0       0.76      0.91      0.83     28023
           1       0.59      0.31      0.40     11415

    accuracy                           0.74     39438
   macro avg       0.68      0.61      0.62     39438
weighted avg       0.71      0.74      0.71     39438

----------------------------------------
--- Training CatBoost Classifier (Patient-Aware Split) ---
CatBoost Accuracy: 0.7380
              precision    recall  f1-score   support

           0       0.76      0.91      0.83     28023
           1       0.59      0.31      0.41     11415

    accuracy                           0.74     39438
   macro avg       0.68      0.61      0.62     39438
weighted avg       0.71      0.74      0.71     39438

----------------------------------------

--- 🏆 Model Performance Summary (Patient-Aware) ---
Logistic Regression: 0.7287
Random Forest:       0.6932
XGBoost:             0.7369
CatBoost:        